# Convolutional Neural Network

### Part 1 - Building the CNN

#### Importing the Tensorflow libraries and packages

In [1]:
import sys

import tensorflow.keras
import pandas as pd
import sklearn as sk
import tensorflow as tf

print(f"Tensor Flow Version: {tf.__version__}")
print(f"Keras Version: {tensorflow.keras.__version__}")
print()
print(f"Python {sys.version}")
print(f"Pandas {pd.__version__}")
print(f"Scikit-Learn {sk.__version__}")
gpu = len(tf.config.list_physical_devices('GPU'))>0
print("GPU is", "available" if gpu else "NOT AVAILABLE")

Tensor Flow Version: 2.6.0
Keras Version: 2.6.0

Python 3.8.0 (default, Nov  6 2019, 16:00:02) [MSC v.1916 64 bit (AMD64)]
Pandas 1.2.5
Scikit-Learn 1.0.2
GPU is available


In [2]:
# Load the TensorBoard notebook extension
%load_ext tensorboard
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import ReduceLROnPlateau

import os

In [4]:
classifier = Sequential()

#classes = 2

#Kernels usually 1x1 | 3x3 | 5x5

#Adding a convolutional layer
classifier.add(Conv2D(16, (3, 3), input_shape = (150, 150, 3), padding='same', activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

#Adding a second convolutional layer
classifier.add(Conv2D(32, (3, 3), padding='same', activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

#Flattening
classifier.add(Flatten())


#Full connection
classifier.add(Dense(units = 32, activation = 'relu'))
classifier.add(Dropout(0.10))
classifier.add(Dense(units = 1, activation= 'sigmoid'))

Let's double check the model description:

In [5]:
classifier.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 150, 150, 16)      448       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 75, 75, 16)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 75, 75, 32)        4640      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 37, 37, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 43808)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 32)                1401888   
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)               

In [6]:
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Part 2 - Fitting the CNN to the images


In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

path_dir = '../dataset_mel_final/train'
parth_dir_test = '../dataset_mel_final/test'

datagen = ImageDataGenerator(
                    rescale=1./255,
                    validation_split = 0.2)

train_generator = datagen.flow_from_directory(
    path_dir,
    target_size=(150,150),
    shuffle=True,
    subset='training',
    batch_size = 64,
    class_mode = 'binary'
)
validation_generator = datagen.flow_from_directory(
    path_dir,
    target_size=(150,150),
    subset='validation',
    batch_size = 64,
    class_mode = 'binary'
)

test_set = datagen.flow_from_directory(parth_dir_test,
                                        target_size= (150,150),
                                        batch_size = 64,
                                        class_mode = 'binary')


Found 1102 images belonging to 2 classes.
Found 274 images belonging to 2 classes.
Found 344 images belonging to 2 classes.


In [8]:
train_generator.class_indices

{'nonviolence': 0, 'violence': 1}

In [9]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
import datetime

#rm -rf /logs/

es = EarlyStopping(monitor='val_loss', patience=10)
mc = ModelCheckpoint('best_cnn.h5', monitor='val_loss', mode='min', save_best_only=True)
reduceLR = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=20)

# Create a TensorBoard callback
log_dir = "logs\\fit\\" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [10]:
classifier.fit(train_generator,
                         epochs = 600,
                         callbacks = [tensorboard_callback,es,mc,reduceLR],
                         validation_data = validation_generator
                         )

Epoch 1/600
18/18 [==============================] - 7s 236ms/step - loss: 1.0051 - accuracy: 0.5045 - val_loss: 0.6937 - val_accuracy: 0.5000
Epoch 2/600
18/18 [==============================] - 4s 207ms/step - loss: 0.6938 - accuracy: 0.5000 - val_loss: 0.6888 - val_accuracy: 0.5000
Epoch 3/600
18/18 [==============================] - 4s 204ms/step - loss: 0.6850 - accuracy: 0.5000 - val_loss: 0.6751 - val_accuracy: 0.5000
Epoch 4/600
18/18 [==============================] - 4s 211ms/step - loss: 0.6604 - accuracy: 0.5753 - val_loss: 0.6489 - val_accuracy: 0.5255
Epoch 5/600
18/18 [==============================] - 4s 216ms/step - loss: 0.6212 - accuracy: 0.6751 - val_loss: 0.5254 - val_accuracy: 0.7774
Epoch 6/600
18/18 [==============================] - 4s 212ms/step - loss: 0.5372 - accuracy: 0.7414 - val_loss: 0.4279 - val_accuracy: 0.8650
Epoch 7/600
18/18 [==============================] - 4s 226ms/step - loss: 0.4249 - accuracy: 0.8094 - val_loss: 0.4978 - val_accuracy: 0.6861

In [11]:
#Overfitting -> Your training accuracy will be higher than the accuracy on the validation/test set

#Overfitting indicates that your model is too complex for the problem that it is solving,
#filters in the case of Convolutional Neural Networks, and layers in the case of overall Deep Learning Models

#How do you know if your model is underfitting? Your model is underfitting if the accuracy on the validation set 
#is higher than the accuracy on the training set. Additionally, 
#if the whole model performs bad this is also called underfitting.

#https://towardsdatascience.com/deep-learning-3-more-on-cnns-handling-overfitting-2bd5d99abe5d

In [15]:
%tensorboard --logdir logs/fit

# Part 3 test the CNN

In [12]:
# Load Model
classifier.load_weights('best_cnn.h5')

In [13]:
classifier.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 150, 150, 16)      448       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 75, 75, 16)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 75, 75, 32)        4640      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 37, 37, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 43808)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 32)                1401888   
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)               

In [14]:
score = classifier.evaluate(test_set, verbose=0)

print('Test Loss', score[0])
print('Test accuracy', score[1])

Test Loss 0.21285784244537354
Test accuracy 0.9244186282157898
